In [ ]:
from google.colab import drive
drive.mount('/content/drive') #to access drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mittens
!pip install gingerit
!pip install tika
!pip install sentence-transformers 

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
from io import StringIO

stemmer = WordNetLemmatizer()
en_stop = set(nltk.corpus.stopwords.words('english'))



from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import csv

import pandas as pd
import numpy as np

from gingerit.gingerit import GingerIt
# pip install tika
from tika import parser
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def clean_document(page):
    page = re.sub(r'[\t]',' ',page)
    page = re.sub(r'[\n]',' ',page)
    page = re.sub(r'[^.,a-zA-Z0-9 \n\.]',' ',page)
    page = re.sub(r'\s+',' ',page)
    return page

In [ ]:
import glob, os

path = '/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/journals'

def readfiles():
    os.chdir(path)
    pdfs = []
    for file in glob.glob("*.pdf"):
        pdfs.append(file)
    return pdfs

In [ ]:
def preprocess_text(document):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(document))
    document = re.sub(r'[^A-Za-z0-9]+',' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    #document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    #document = re.sub(r'^b\s+', '', document)
    #document = re.sub()
    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    tokens = document.split()
    #tokens = [stemmer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in en_stop]
    tokens = [word for word in tokens if len(word) > 3]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
def sentance_correction(corpus):
    textCorrected =[]
    for i in corpus:
        try:
            parser = GingerIt()
            text = parser.parse(i)['result']
            textCorrected.append(text)
        except:
            continue
    
    return textCorrected

In [ ]:
files = readfiles()

In [ ]:
files

['kundu-2015-a.pdf',
 'An Analysis of Overlapping Community Detection Algorithms in Social Networks.pdf']

In [ ]:
files = readfiles()

df_all = []
for file in files:
    file_name = file #'ICDE_workshopv20.pdf'
    raw = parser.from_file(file_name)
    print(file)
    corpus = [clean_document(raw['content'])] #word form
    corpus_doc = [' '.join(corpus)] #sentence form

    stemmer = WordNetLemmatizer()

    en_stop = set(nltk.corpus.stopwords.words('english'))

    sentence = []
    for s in corpus_doc:
        sentence.append(sent_tokenize(s)) #paragraph


    textCorrected = sentance_correction(sentence[0])
    corpus = [preprocess_text(i) for i in textCorrected]

    df = pd.DataFrame()
    df['text'] = corpus
    df['correct_sentances'] = textCorrected
    df['file_name'] = file_name
    file = df

    df_all.append(file)

kundu-2015-a.pdf
An Analysis of Overlapping Community Detection Algorithms in Social Networks.pdf


In [ ]:
df = pd.concat(df_all)

In [ ]:
df

,text,correct_sentances,file_name
0,pattern recognition letters journal homepage e...,1 Pattern Recognition Letters journal homepag...,kundu-2015-a.pdf
1,algorithms available literature mainly follow ...,The algorithms available in the literature mai...,kundu-2015-a.pdf
2,paper proposed novel community detection algor...,"In this paper, we proposed a novel community d...",kundu-2015-a.pdf
3,algorithm runs framework social network repres...,The algorithm runs on a new framework of socia...,kundu-2015-a.pdf
4,index,A new index viz.,kundu-2015-a.pdf
...,...,...,...
346,2007,"036 106, 2007 .",An Analysis of Overlapping Community Detection...
347,lancichinetti fortunato kertesz detecting over...,"24 A. Lancichinetti, S. Fortunato and J. Kerte...",An Analysis of Overlapping Community Detection...
348,033015 2009,"11, 033015, p. 18, 2009.",An Analysis of Overlapping Community Detection...
349,qiguo maozu yang xiaoyan ling chen mlpa detect...,"25 Qiguo Dai, MaoZu Guo, Yang Liu, Xiaoyan Liu...",An Analysis of Overlapping Community Detection...


In [ ]:
query = input('enter the query:')
query = preprocess_text(query)
parser = GingerIt()
query = parser.parse(query)['result']

enter the query:modularity is in the community detction?


In [ ]:
query

'modularity community detection'

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation')

from Embedding_models import glove50_embedding
from Embedding_models import Bert_embedding
from Embedding_models import elmo_embedding

glove_model = glove50_embedding(df)
glove_model.fit()

# Save the model as a pickle in a file
joblib.dump(glove_model, '/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/glove_trained.pkl')

['/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/glove_trained.pkl']

In [ ]:
# Load the model from the file
glove_model_saved = joblib.load('/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/glove_trained.pkl')

df_glove = glove_model_saved.fit_transform(query)

/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/Embedding_models.py:112: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = nominator / denominator


In [ ]:
df_glove.iloc[:5,1:]

,correct_sentances,file_name,cosine_score
0,Modularity Approaches in Community Detection V...,An Analysis of Overlapping Community Detection...,0.877165
1,Overview of Modularity Adaptation based Commun...,An Analysis of Overlapping Community Detection...,0.768273
2,It gives the review of modularity adaptation b...,An Analysis of Overlapping Community Detection...,0.738082
3,Section 3 explains the modularity based commun...,An Analysis of Overlapping Community Detection...,0.737236
4,This work aims at providing the characteristic...,An Analysis of Overlapping Community Detection...,0.727032


In [ ]:
list(df_glove['cosine_score'].head(10))

[0.8771647214889526,
 0.7682733535766602,
 0.7380821108818054,
 0.7372363805770874,
 0.7270323634147644,
 0.6977030634880066,
 0.6686495246921579,
 0.6577390432357788,
 0.6492091417312622,
 0.639761209487915]

In [ ]:
bert_model = Bert_embedding(df)
bert_model.fit()

 
# Save the model as a pickle in a file
joblib.dump(bert_model, '/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/bert_trained.pkl')

['/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/bert_trained.pkl']

In [ ]:
# Load the model from the file
bert_model_saved = joblib.load('/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/bert_trained.pkl')

df_bert = bert_model_saved.fit_transform(query)

In [ ]:
df_bert.iloc[:5,1:]

,correct_sentances,file_name,cosine_score
0,Overview of Modularity Adaptation based Commun...,An Analysis of Overlapping Community Detection...,0.931679
1,Section 3 explains the modularity based commun...,An Analysis of Overlapping Community Detection...,0.899921
2,It gives the review of modularity adaptation b...,An Analysis of Overlapping Community Detection...,0.848959
3,This modularity is the strength of partition o...,An Analysis of Overlapping Community Detection...,0.845770
4,Modularity defines the strength of partition o...,An Analysis of Overlapping Community Detection...,0.839639


In [ ]:
list(df_bert['correct_sentances'].head(10))

['Overview of Modularity Adaptation based Community Detection.',
 'Section 3 explains the modularity based community detection approach in detail.',
 'It gives the review of modularity adaptation based community detection algorithms.',
 'This modularity is the strength of partition of the network as a community.',
 'Modularity defines the strength of partition of network into communities.',
 'Modularity Approaches in Community Detection Various algorithms of community detection are analyzed based on modularity.',
 'The modularity value of the community structure formed in the Fig.',
 'Definition 8 Community.',
 'Summary of Modularity Score.',
 'It is used in modularity function to find the strength of community.']

In [ ]:
from Embedding_models import elmo_embedding

elmo_model = elmo_embedding(df)
elmo_model.fit()

# Save the model as a pickle in a file
joblib.dump(glove_model, '/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/elmo_trained.pkl')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# Load the model from the file
elmo_model_saved = joblib.load('/content/drive/MyDrive/Classroom/2021 Machine Learning 1 (CSL7550)/Project/Implementation/elmo_trained.pkl')

df_elmo = elmo_model_saved.fit_transform(query)

In [ ]:
df_elmo.iloc[:5,1:]

In [ ]:
list(df_elmo['correct_sentances'].head(10))